# Property Usage in Classes and Structs - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **trivial** implementations of properties in classes and structs.

## Properties in Classes

### ✅ Good Examples - Meaningful Property Usage in Classes

In [ ]:
// ✅ GOOD: Properties with validation and business logic
public class BankAccount
{
    private decimal _balance;
    private string _accountNumber;
    private List<string> _transactionHistory = new();
    
    // Property with validation
    public decimal Balance 
    { 
        get => _balance;
        private set // Only allow internal modification
        {
            if (value < 0)
                throw new InvalidOperationException("Balance cannot be negative");
            _balance = value;
        }
    }
    
    // Property with format validation
    public string AccountNumber
    {
        get => _accountNumber;
        set
        {
            if (string.IsNullOrEmpty(value) || value.Length != 10)
                throw new ArgumentException("Account number must be 10 digits");
            if (!value.All(char.IsDigit))
                throw new ArgumentException("Account number must contain only digits");
            _accountNumber = value;
        }
    }
    
    // Computed property
    public string FormattedAccountNumber => 
        $"{AccountNumber?.Substring(0, 3)}-{AccountNumber?.Substring(3, 3)}-{AccountNumber?.Substring(6)}";
    
    // Read-only property based on state
    public bool IsOverdrawn => Balance < 0;
    
    // Property that triggers side effects
    public bool IsActive { get; set; } = true;
    
    // Indexer property for transaction history
    public string this[int index]
    {
        get
        {
            if (index < 0 || index >= _transactionHistory.Count)
                throw new IndexOutOfRangeException("Invalid transaction index");
            return _transactionHistory[index];
        }
    }
    
    // Property with lazy initialization
    private string _accountType;
    public string AccountType
    {
        get
        {
            if (_accountType == null)
            {
                _accountType = Balance >= 10000 ? "Premium" : "Standard";
            }
            return _accountType;
        }
    }
    
    public void Deposit(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Deposit amount must be positive");
            
        Balance += amount; // Uses property validation
        _transactionHistory.Add($"Deposit: +{amount:C} at {DateTime.Now}");
        _accountType = null; // Reset lazy property
    }
    
    public void Withdraw(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Withdrawal amount must be positive");
        if (Balance - amount < -1000) // Overdraft limit
            throw new InvalidOperationException("Insufficient funds");
            
        Balance -= amount;
        _transactionHistory.Add($"Withdrawal: -{amount:C} at {DateTime.Now}");
        _accountType = null; // Reset lazy property
    }
}

In [ ]:
// Usage example
var account = new BankAccount { AccountNumber = "1234567890" };
account.Deposit(1500.50m);
Console.WriteLine($"Balance: {account.Balance:C}");
Console.WriteLine($"Formatted Account: {account.FormattedAccountNumber}");
Console.WriteLine($"Account Type: {account.AccountType}");
Console.WriteLine($"Last Transaction: {account[0]}");

In [ ]:
// ✅ GOOD: Properties with change notification (common in UI applications)
public class Product : INotifyPropertyChanged
{
    private string _name;
    private decimal _price;
    private int _stock;
    
    public event PropertyChangedEventHandler PropertyChanged;
    
    public string Name
    {
        get => _name;
        set
        {
            if (_name != value)
            {
                _name = value;
                OnPropertyChanged();
                OnPropertyChanged(nameof(DisplayName)); // Notify dependent property
            }
        }
    }
    
    public decimal Price
    {
        get => _price;
        set
        {
            if (value < 0)
                throw new ArgumentException("Price cannot be negative");
            if (_price != value)
            {
                _price = value;
                OnPropertyChanged();
                OnPropertyChanged(nameof(DisplayPrice));
                OnPropertyChanged(nameof(IsExpensive));
            }
        }
    }
    
    public int Stock
    {
        get => _stock;
        set
        {
            if (value < 0)
                throw new ArgumentException("Stock cannot be negative");
            if (_stock != value)
            {
                _stock = value;
                OnPropertyChanged();
                OnPropertyChanged(nameof(IsInStock));
                OnPropertyChanged(nameof(StockStatus));
            }
        }
    }
    
    // Computed properties that depend on other properties
    public string DisplayName => $"{Name} (#{GetHashCode() % 1000:D3})";
    public string DisplayPrice => $"{Price:C}";
    public bool IsInStock => Stock > 0;
    public bool IsExpensive => Price > 100;
    
    public string StockStatus => Stock switch
    {
        0 => "Out of Stock",
        < 5 => "Low Stock",
        < 20 => "In Stock",
        _ => "Well Stocked"
    };
    
    private void OnPropertyChanged([CallerMemberName] string propertyName = null)
    {
        PropertyChanged?.Invoke(this, new PropertyChangedEventArgs(propertyName));
    }
}

In [ ]:
using System.ComponentModel;
using System.Runtime.CompilerServices;

// Usage example with event handling
var product = new Product();
product.PropertyChanged += (sender, e) => 
    Console.WriteLine($"Property {e.PropertyName} changed");
    
product.Name = "Laptop";
product.Price = 999.99m;
product.Stock = 3;

Console.WriteLine($"Display: {product.DisplayName}");
Console.WriteLine($"Stock Status: {product.StockStatus}");
Console.WriteLine($"Is Expensive: {product.IsExpensive}");

### ❌ Bad Examples - Trivial Property Usage in Classes

In [ ]:
// ❌ BAD: Just auto-implemented properties with no validation or logic
public class BadPerson
{
    public string Name { get; set; }
    public int Age { get; set; }
    public string Email { get; set; }
    public string Address { get; set; }
}

// Problems:
// 1. No validation (age could be negative, email format not checked)
// 2. No encapsulation (direct field access would be the same)
// 3. No business logic
// 4. Could be a record instead

// ❌ BAD: Property that just wraps a field with no added value
public class BadExample
{
    private string _value;
    
    public string Value
    {
        get { return _value; }
        set { _value = value; }
    }
    
    // This is pointless - just use auto-property or public field
    // No validation, no logic, no encapsulation benefit
}

## Properties in Structs

### ✅ Good Examples - Meaningful Property Usage in Structs

In [ ]:
// ✅ GOOD: Immutable struct with computed properties
public readonly struct Temperature : IComparable<Temperature>, IEquatable<Temperature>
{
    private readonly double _celsius;
    
    // Primary property with validation
    public double Celsius 
    { 
        get => _celsius;
        init // Only settable during initialization
        {
            if (value < -273.15)
                throw new ArgumentException("Temperature cannot be below absolute zero");
            _celsius = value;
        }
    }
    
    // Computed properties for different temperature scales
    public double Fahrenheit => (Celsius * 9.0 / 5.0) + 32.0;
    public double Kelvin => Celsius + 273.15;
    public double Rankine => Kelvin * 9.0 / 5.0;
    
    // Properties that provide meaningful information
    public bool IsFreezing => Celsius <= 0;
    public bool IsBoiling => Celsius >= 100;
    public bool IsRoomTemperature => Celsius >= 18 && Celsius <= 24;
    
    public string Scale => "Celsius";
    
    // Property that categorizes the temperature
    public string Category => Celsius switch
    {
        < -20 => "Extremely Cold",
        < 0 => "Very Cold",
        < 10 => "Cold",
        < 20 => "Cool",
        < 30 => "Warm",
        < 40 => "Hot",
        _ => "Extremely Hot"
    };
    
    public Temperature(double celsius)
    {
        Celsius = celsius; // Uses property validation
    }
    
    // Factory methods using properties
    public static Temperature FromFahrenheit(double fahrenheit) => 
        new((fahrenheit - 32.0) * 5.0 / 9.0);
    
    public static Temperature FromKelvin(double kelvin) => 
        new(kelvin - 273.15);
    
    // Equality and comparison using property
    public bool Equals(Temperature other) => Math.Abs(Celsius - other.Celsius) < 0.01;
    public override bool Equals(object obj) => obj is Temperature other && Equals(other);
    public override int GetHashCode() => Celsius.GetHashCode();
    
    public int CompareTo(Temperature other) => Celsius.CompareTo(other.Celsius);
    
    public override string ToString() => $"{Celsius:F1}°C ({Category})";
}

In [ ]:
// Usage examples
var temp1 = new Temperature(25.0);
var temp2 = Temperature.FromFahrenheit(77.0);

Console.WriteLine($"Temperature: {temp1}");
Console.WriteLine($"Fahrenheit: {temp1.Fahrenheit:F1}°F");
Console.WriteLine($"Kelvin: {temp1.Kelvin:F1}K");
Console.WriteLine($"Is room temperature: {temp1.IsRoomTemperature}");
Console.WriteLine($"Category: {temp1.Category}");
Console.WriteLine($"Equal temps: {temp1.Equals(temp2)}");

In [ ]:
// ✅ GOOD: Point struct with meaningful properties and validation
public readonly struct Point2D : IEquatable<Point2D>
{
    public double X { get; init; }
    public double Y { get; init; }
    
    // Computed properties for geometric calculations
    public double DistanceFromOrigin => Math.Sqrt(X * X + Y * Y);
    public double Angle => Math.Atan2(Y, X) * 180.0 / Math.PI;
    
    // Properties that provide meaningful geometric information
    public bool IsOrigin => X == 0 && Y == 0;
    public bool IsOnXAxis => Y == 0;
    public bool IsOnYAxis => X == 0;
    
    public Quadrant Quadrant => (X, Y) switch
    {
        (> 0, > 0) => Quadrant.First,
        (< 0, > 0) => Quadrant.Second,
        (< 0, < 0) => Quadrant.Third,
        (> 0, < 0) => Quadrant.Fourth,
        _ => Quadrant.OnAxis
    };
    
    // Indexer property for coordinate access
    public double this[int index] => index switch
    {
        0 => X,
        1 => Y,
        _ => throw new IndexOutOfRangeException("Point2D only has X(0) and Y(1) coordinates")
    };
    
    public Point2D(double x, double y)
    {
        X = x;
        Y = y;
    }
    
    // Methods using properties
    public double DistanceTo(Point2D other)
    {
        var dx = X - other.X;
        var dy = Y - other.Y;
        return Math.Sqrt(dx * dx + dy * dy);
    }
    
    public Point2D Translate(double dx, double dy) => new(X + dx, Y + dy);
    public Point2D Scale(double factor) => new(X * factor, Y * factor);
    public Point2D Rotate(double degrees)
    {
        var radians = degrees * Math.PI / 180.0;
        var cos = Math.Cos(radians);
        var sin = Math.Sin(radians);
        return new Point2D(X * cos - Y * sin, X * sin + Y * cos);
    }
    
    public bool Equals(Point2D other) => 
        Math.Abs(X - other.X) < 1e-10 && Math.Abs(Y - other.Y) < 1e-10;
    
    public override bool Equals(object obj) => obj is Point2D other && Equals(other);
    public override int GetHashCode() => HashCode.Combine(X, Y);
    public override string ToString() => $"({X:F2}, {Y:F2}) [{Quadrant}]";
}

public enum Quadrant { First, Second, Third, Fourth, OnAxis }



In [ ]:
// Usage examples
var point = new Point2D(3.0, 4.0);
Console.WriteLine($"Point: {point}");
Console.WriteLine($"Distance from origin: {point.DistanceFromOrigin:F2}");
Console.WriteLine($"Angle: {point.Angle:F1}°");
Console.WriteLine($"Quadrant: {point.Quadrant}");
Console.WriteLine($"X coordinate: {point[0]}");

var rotated = point.Rotate(90);
Console.WriteLine($"Rotated 90°: {rotated}");

### ❌ Bad Examples - Trivial Property Usage in Structs

In [ ]:
// ❌ BAD: Mutable struct with simple auto-properties
public struct BadPoint
{
    public int X { get; set; }
    public int Y { get; set; }
}

// Problems:
// 1. Mutable struct (can cause confusing behavior)
// 2. No validation
// 3. No meaningful computed properties
// 4. No business logic
// 5. Could just be two separate variables

// ❌ BAD: Struct that's too complex and should be a class
public struct BadComplexStruct
{
    public string Name { get; set; }
    public string Description { get; set; }
    public List<string> Tags { get; set; } // Reference type!
    public Dictionary<string, object> Properties { get; set; } // Reference type!
    
    // Too large, contains reference types, should be a class
}

// ❌ BAD: Properties that just expose fields without any added value
public struct BadExample
{
    private int _value;
    
    public int Value 
    {
        get => _value;
        set => _value = value;
    }
    
    // Pointless - just use auto-property or public field
    // No validation, computation, or meaningful behavior
}

## Advanced Property Patterns

### ✅ Good Examples - Advanced Meaningful Property Usage

In [ ]:
// ✅ GOOD: Properties with caching and expensive calculations
public class DataProcessor
{
    private List<double> _data = new();
    private double? _cachedAverage;
    private double? _cachedStandardDeviation;
    private bool _isDirty = true;
    
    public IReadOnlyList<double> Data => _data.AsReadOnly();
    
    // Property with expensive calculation and caching
    public double Average
    {
        get
        {
            if (_isDirty || !_cachedAverage.HasValue)
            {
                _cachedAverage = _data.Count > 0 ? _data.Average() : 0;
                Console.WriteLine("Average calculated"); // For demo
            }
            return _cachedAverage.Value;
        }
    }
    
    // Another cached property
    public double StandardDeviation
    {
        get
        {
            if (_isDirty || !_cachedStandardDeviation.HasValue)
            {
                if (_data.Count == 0)
                {
                    _cachedStandardDeviation = 0;
                }
                else
                {
                    var avg = Average; // Uses cached value
                    var variance = _data.Select(x => Math.Pow(x - avg, 2)).Average();
                    _cachedStandardDeviation = Math.Sqrt(variance);
                }
                Console.WriteLine("Standard deviation calculated"); // For demo
            }
            return _cachedStandardDeviation.Value;
        }
    }
    
    // Properties that depend on cached calculations
    public double Minimum => _data.Count > 0 ? _data.Min() : 0;
    public double Maximum => _data.Count > 0 ? _data.Max() : 0;
    public double Range => Maximum - Minimum;
    public int Count => _data.Count;
    public bool IsEmpty => _data.Count == 0;
    
    // Indexer with bounds checking
    public double this[int index]
    {
        get
        {
            if (index < 0 || index >= _data.Count)
                throw new IndexOutOfRangeException($"Index {index} is out of range [0, {_data.Count - 1}]");
            return _data[index];
        }
        set
        {
            if (index < 0 || index >= _data.Count)
                throw new IndexOutOfRangeException($"Index {index} is out of range [0, {_data.Count - 1}]");
            _data[index] = value;
            InvalidateCache();
        }
    }
    
    public void AddValue(double value)
    {
        _data.Add(value);
        InvalidateCache();
    }
    
    public void Clear()
    {
        _data.Clear();
        InvalidateCache();
    }
    
    private void InvalidateCache()
    {
        _isDirty = true;
        _cachedAverage = null;
        _cachedStandardDeviation = null;
    }
}



In [ ]:
// Usage example
var processor = new DataProcessor();
processor.AddValue(10.0);
processor.AddValue(20.0);
processor.AddValue(30.0);

Console.WriteLine($"Average: {processor.Average}"); // Calculates
Console.WriteLine($"Average: {processor.Average}"); // Uses cache
Console.WriteLine($"Std Dev: {processor.StandardDeviation}"); // Calculates
Console.WriteLine($"Range: {processor.Range}");
Console.WriteLine($"Value at index 1: {processor[1]}");

processor[1] = 25.0; // Invalidates cache
Console.WriteLine($"New Average: {processor.Average}"); // Recalculates

## Summary

### What Makes Property Usage "Meaningful":

**In Classes:**
- **Validation**: Check input values before setting
- **Computed Properties**: Calculate values from other properties
- **Change Notification**: Notify when properties change (INotifyPropertyChanged)
- **Encapsulation**: Control access to internal state
- **Side Effects**: Trigger actions when values change
- **Lazy Initialization**: Calculate expensive values only when needed
- **Caching**: Store calculated values until data changes
- **Indexers**: Provide array-like access with validation

**In Structs:**
- **Immutability**: Use `readonly` and `init` properties
- **Computed Values**: Calculate related values (temperature conversions, geometric properties)
- **Validation**: Ensure value types maintain valid state
- **Value Semantics**: Properties that make sense for value comparison
- **Small and Focused**: Properties that support the struct's single responsibility

### Red Flags (Avoid These):

**Bad Property Patterns:**
- Simple auto-properties with no validation or logic
- Properties that just wrap fields without added value
- Mutable structs with setter properties
- Large structs with many properties (should be classes)
- Properties with reference types in structs
- Properties that are never used in business logic
- Missing validation for properties that need it
- Not invalidating caches when dependent data changes

### Property Types to Demonstrate:

1. **Auto-Implemented Properties** - Simple cases with basic validation
2. **Full Properties** - With backing fields and complex logic
3. **Computed Properties** - Calculate values from other properties
4. **Indexer Properties** - Array-like access with validation
5. **Init-Only Properties** - For immutable objects
6. **Read-Only Properties** - Computed or cached values
7. **Properties with Change Notification** - For UI binding scenarios
8. **Cached Properties** - For expensive calculations